In [19]:
import pandas as pd
import cerberus
import numpy as np

In [31]:
def parse_meta(species):
    df = pd.read_csv(f'data/{species}/metadata.tsv', sep='\t')
    df['species'] = species
    df['subset'] = subset
    df['dataset'] = df['tissue']+'_'+\
                 df['age']+'_'+\
                 df['platform']+'_'+\
                 df['capture']+'_'+\
                 df['species']+'_'+\
                 df['subset']
    
    return df

In [56]:
DEBUG = True

## Capture vs. non capture triplets

In [60]:
species = 'human'
obs_col = 'capture'
min_tpm = 1

ca = cerberus.read(f'data/{species}/cerberus/ca_trip.h5')
df = pd.read_csv(f'data/{species}/cerberus/agg_ab.tsv', sep='\t')
df.drop('annot_transcript_name', axis=1, inplace=True)
df.set_index('annot_transcript_id', inplace=True)
df = df.transpose()
df = df.fillna(0)
meta = parse_meta(species)
meta_cols = meta.columns
df = df.merge(meta, how='left', left_index=True, right_on='dataset')
df.drop([c for c in meta_cols if c != obs_col], axis=1, inplace=True)

# groupby and take max tpm value per thing
df = df.groupby('capture').max().reset_index()

# loop through the different conditions
trip = pd.DataFrame()
for obs_cond in df[obs_col].unique():
    source = f'{obs_col}_{obs_cond}'
    temp = df.loc[df[obs_col] == obs_cond].copy(deep=True)
    temp = temp.drop(obs_col, axis=1).transpose()
    temp.columns = ['max_tpm']
    if DEBUG: print(len(temp.index))
    temp = temp.loc[temp.max_tpm >= min_tpm]
    if DEBUG: print(f'{len(temp.index)}\n')
    tids = temp.index.tolist()
    temp_trip = ca.get_subset_triplets(tids, source)
    trip = pd.concat([trip, temp_trip], axis=0)
    

523659
372434

523659
245329



In [61]:
ca.triplets.head()

,source


In [55]:
temp = temp.drop(obs_col, axis=1).transpose()
temp.columns = ['max_tpm']
print(len(temp.index))
temp = temp.loc[temp.max_tpm >= min_tpm]
print(len(temp.index))
tids = temp.index.tolist()

523659
245329


In [24]:
# # ca.triplets.head()
# df = pd.read_csv('data/human/cerberus/triplets.tsv', sep='\t')
# meta = pd.read_csv('data/human/metadata.tsv', sep='\t')
# meta['species'] = species
# meta['subset'] = subset
# meta['dataset'] = meta['tissue']+'_'+\
#              meta['age']+'_'+\
#              meta['platform']+'_'+\
#              meta['capture']+'_'+\
#              meta['species']+'_'+\
#              meta['subset']